In [ ]:

from matplotlib import pyplot
import pandas as pd

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_validate, KFold
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB


In [ ]:

data = pd.read_csv("data.csv")
data1 = pd.read_csv("data.csv")
data



In [ ]:
data.dtypes

In [ ]:
data.info()

In [ ]:

print("Number Of Rows In The Original DataFrame:", len(data))
print("Number Of Rows After Deduping:", len(data.drop_duplicates()))

'''
Diagnosis - specific combination of ICD-10 codes for primary and secondary as well multiple tumours if any		
PRE4	Feature	Continuous		Forced vital capacity - FVC (numeric)	
PRE5	Feature	Continuous		Volume that has been exhaled at the end of the first second of forced expiration - FEV1 (numeric)		
PRE6	Feature	Categorical		Performance status - Zubrod scale (PRZ2,PRZ1,PRZ0)		
PRE7	Feature	Binary		Pain before surgery (T,F)	
PRE8	Feature	Binary		Haemoptysis before surgery (T,F)		
PRE9	Feature	Binary		Dyspnoea before surgery (T,F)		
PRE10	Feature	Binary		Cough before surgery (T,F)	
PRE11	Feature	Binary		Weakness before surgery (T,F)	
PRE14	Feature	Categorical		T in clinical TNM - size of the original tumour, from OC11 (smallest) to OC14 (largest) (OC11,OC14,OC12,OC13)	
PRE17	Feature	Binary		Type 2 DM - diabetes mellitus (T,F)		no
PRE19	Feature	Binary		MI up to 6 months (T,F)		no
PRE25	Feature	Binary		PAD - peripheral arterial diseases (T,F)		no
PRE30	Feature	Binary		Smoking (T,F)		no
PRE32	Feature	Binary		Asthma (T,F)		no
AGE	Feature	Integer	Age	Age at surgery (numeric)		no
Risk1Yr	Target	Binary		1 year survival period - (T)rue value if died (T,F)		no
'''

In [ ]:
data.describe(include = "all")

In [ ]:


dgn = data['DGN']
pre6 = data['PRE6']
pre14 = data['PRE14']

pyplot.figure()
pyplot.subplot(311);pyplot.xlabel('dgn'); pyplot.hist(dgn)
pyplot.subplot(312); pyplot.xlabel('pre6'); pyplot.hist(pre6) 
pyplot.subplot(313);pyplot.xlabel('pre14'); pyplot.hist(pre14) 
# pyplot.show()

age = data['AGE']
pyplot.xlabel('age'); pyplot.hist(age) ; pyplot.show()

pre4 = data['PRE4']
pyplot.xlabel('Forced vital capacity - FVC'); pyplot.hist(pre4);pyplot.show()

pre5 = data['PRE5']
pyplot.xlabel('Volume that has been exhaled at the end of the first second of forced expiration'); pyplot.hist(pre5); pyplot.show()

pre7 = data['PRE7']
pre8 = data['PRE8']
pre9 = data['PRE9']
pre10 = data['PRE10']

pyplot.figure()
pyplot.subplot(221); pyplot.xlabel('pre10'); pyplot.hist(pre10) 
pyplot.subplot(222);pyplot.xlabel('pre7'); pyplot.hist(pre7)
pyplot.subplot(223); pyplot.xlabel('pre8'); pyplot.hist(pre8) 
pyplot.subplot(224);pyplot.xlabel('pre9'); pyplot.hist(pre9)
# pyplot.show()


pre11 = data['PRE11']
pre17 = data['PRE17']
pre19 = data['PRE19']

pyplot.figure()
pyplot.subplot(311);pyplot.xlabel('pre11'); pyplot.hist(pre11)
pyplot.subplot(312); pyplot.xlabel('pre17'); pyplot.hist(pre17) 
pyplot.subplot(313);pyplot.xlabel('pre19'); pyplot.hist(pre19)
pyplot.show()

pre25 = data['PRE25']
pre30 = data['PRE30']
pre32 = data['PRE32']

pyplot.figure()
pyplot.subplot(311);pyplot.ylabel('pre25'); pyplot.hist(pre25)
pyplot.subplot(312);pyplot.ylabel('pre30'); pyplot.hist(pre30)
pyplot.subplot(313);pyplot.ylabel('pre32'); pyplot.hist(pre32)
pyplot.show()




In [ ]:
for column in data.columns[:-2]:
    pyplot.figure(figsize=(5, 3))
    sns.histplot(data[column], kde=True)
    pyplot.title(f'Distribution of {column}')
    pyplot.show()

In [ ]:
filtered_data = data1.loc[data1['PRE25'] == 'T', 'Risk1Yr']

filtered_data


In [ ]:
# data1 = data1.drop(columns=['PRE32', 'PRE19', 'PRE25'])
# data1

In [ ]:
labels = ['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17','PRE30','Risk1Yr','PRE32', 'PRE19', 'PRE25']
data2 = pd.get_dummies(data=data1, columns=['DGN'], dtype=int)

data2 = pd.get_dummies(data=data2, columns=labels, dtype=int, drop_first=True)

# Create mapper
scale_mapper = {'OC11':1,
 'OC12':2,
 'OC13':3,
 'OC14':4 }

data2["PRE14"] = data2["PRE14"].replace(scale_mapper)

scale_mapper2 = {'PRZ0':1,
 'PRZ1':2,
 'PRZ2':3}

data2["PRE6"] = data2["PRE6"].replace(scale_mapper2)
data2.head()




In [ ]:
df_corr = data2.corr()
fig, ax = pyplot.subplots(figsize = (17,17))
sns.heatmap(data=df_corr, annot=True, ax=ax, cmap='Blues')

In [ ]:
#sve kolone koje nisu konacne
cols = [col for col in data2.columns if col not in ['Risk1Yr_T']]
data_features = data2[cols]
target = data2['Risk1Yr_T']
data_features.head(2)

# Standardize features
standardizer = StandardScaler()
data_features =  standardizer.fit_transform(data_features)


In [ ]:

results = {
    'knn': {
        'basic':0,
        'cross':0,
        'balanced':0
    },
    'random forest': {
        'basic':0,
        'cross':0,
        'balanced':0
    },
    'Logistic Regression':{
        'basic':0,
        'cross':0,
        'balanced':0
    },
    "LinearSVC":{
        'basic':0,
        'cross':0,
        'balanced':0
    },
    "BernoulliNB":{
        'basic':0,
        'cross':0,
        'balanced':0
    },
    'GaussianNB':{
        'basic':0,
        'cross':0,
        'balanced':0
    },
    'DecisionTree':{
        'basic':0,
        'cross':0,
        'balanced':0
    }
}



In [ ]:

data_train,data_test,target_train,target_test = train_test_split(data_features,target,test_size=0.2,random_state=47)

print(f"Broj uzoraka u trening skupu podataka: {len(data_train)}")
print(f"Broj uzoraka u test skupu podataka: {len(data_test)}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
# Create classifier
rf_classifier = RandomForestClassifier()
# Train model
rf_model = rf_classifier.fit(data_train, target_train)
rf_pred = rf_model.predict(data_test)
# Get accuracy score
res = accuracy_score(target_test, rf_pred, normalize = True)
print(" RandomForest accuracy : ", res)

results['random forest']['basic'] = res


In [ ]:
# trazimo znacajne podatke
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
names = [data2.columns[i] for i in indices]

pyplot.figure()

pyplot.title("Feature Importance")

pyplot.bar(range(data_train.shape[1]), importances[indices])

pyplot.xticks(range(data_train.shape[1]), names, rotation=90)

pyplot.show()


In [ ]:

from sklearn.feature_selection import SelectFromModel


def selectFeaturesRF(th):
    selector = SelectFromModel(rf_classifier, threshold=th) 

    features_important = selector.fit_transform(data_train, target_train)

    test_important = selector.transform(data_test)

    randomForestModel = rf_classifier.fit(features_important,target_train)

    predRFC = randomForestModel.predict(test_important)
    res = accuracy_score(target_test, predRFC, normalize = True)
    print("RandomForest accuracy : ",res )
    results['random forest']['importance'] = res

selectFeaturesRF(0.032)



In [ ]:

kfold = KFold(n_splits=10, shuffle=True, random_state=1)


In [ ]:

from sklearn.neighbors import KNeighborsClassifier

#dodati scaler
nearest_neighbors = KNeighborsClassifier(n_neighbors=8)

knn_model = nearest_neighbors.fit(data_train,target_train)

predNN = knn_model.predict(data_test)

res=nearest_neighbors.score(data_test, target_test)
print("KNeighbors accuracy : ", res)

results['knn']['basic'] = res


In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression


logistic_regression = LogisticRegression(random_state=0)
# Train model
lr_model = logistic_regression.fit(data_train, target_train)

pred_lr = lr_model.predict(data_test)

print("Logistic Regression : ",accuracy_score(target_test, pred_lr, normalize = True))

results['Logistic Regression']['basic'] = accuracy_score(target_test, pred_lr, normalize = True)


In [ ]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC( C=1.0, random_state=0,  dual='auto')

pred = svc_model.fit(data_train, target_train).predict(data_test)

print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

results['LinearSVC']['basic'] = accuracy_score(target_test, pred_lr, normalize = True)

In [ ]:
# Naive Bayes
import numpy as np
from sklearn.naive_bayes import BernoulliNB

nb_classifer = BernoulliNB()
model = nb_classifer.fit(data_train, target_train).predict(data_test)

print("BernoulliNB : ",accuracy_score(target_test, model, normalize = True))

results['BernoulliNB']['basic'] = accuracy_score(target_test, model, normalize = True)


In [ ]:

# Inicijalizacija i treniranje modela 
gnb_classifer = GaussianNB()
gnb_classifer.fit(data_train, target_train)

# Predikcija na test skupu
pred = gnb_classifer.predict(data_test)

# Evaluacija performansi modela
print("GaussianNB() accuracy:  ", (accuracy_score(target_test, pred, normalize = True)))
results['GaussianNB']['basic'] = accuracy_score(target_test, pred, normalize = True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decisionTree = DecisionTreeClassifier(random_state=0)

predDT = decisionTree.fit(data_train, target_train).predict(data_test)

print("DecisionTree accuracy : ", accuracy_score(target_test, predDT, normalize = True))

results['DecisionTree']['basic'] = accuracy_score(target_test, predDT, normalize = True)

In [ ]:
from imblearn.over_sampling import SMOTE

data_pre_smote = pd.DataFrame({'Class': target})
class_counts_pre_smote = data_pre_smote['Class'].value_counts()

pyplot.figure(figsize=(5, 3))
pie_pre_smote = class_counts_pre_smote.plot(kind='pie', autopct='%1.0f%%')
pie_pre_smote.figure.set_size_inches(5, 3)
pie_pre_smote.figure.legend(loc='lower right')
pyplot.title("Distribucija klasa pre SMOTE")
pyplot.show()

sm = SMOTE(random_state=2)
data_features_res, target_res = sm.fit_resample(data_features, target)

data_post_smote = pd.DataFrame({'Class': target_res})
class_counts_post_smote = data_post_smote['Class'].value_counts()

pyplot.figure(figsize=(5, 3))
pie_post_smote = class_counts_post_smote.plot(kind='pie', autopct='%1.0f%%')
pie_post_smote.figure.set_size_inches(5, 3)
pie_post_smote.figure.legend(loc='lower right')
pyplot.title("Distribucija klasa posle SMOTE")
pyplot.show()


In [ ]:

data_train, data_test, target_train, target_test = train_test_split(data_features_res, target_res, test_size=0.2, random_state=10)


In [ ]:
svc_model.fit(data_train, target_train)
# Predikcija na test skupu
pred = svc_model.predict(data_test)
# Evaluacija performansi modela
print("LinearSVC accuracy after SMOTE:  ", (accuracy_score(target_test, pred)))


rf_model.fit(data_train, target_train)
rf_pred = rf_model.predict(data_test)
print("random forest accuracy after SMOTE:  ", (accuracy_score(target_test, rf_pred)))

knn_model.fit(data_train, target_train)
predNN = rf_model.predict(data_test)
print("knn accuracy after SMOTE:  ", (accuracy_score(target_test, predNN)))

predDT = decisionTree.fit(data_train, target_train).predict(data_test)
print("decision tree accuracy after SMOTE:  ", (accuracy_score(target_test, predDT)))


BernoulliNB = nb_classifer.fit(data_train, target_train).predict(data_test)
print("nb accuracy after SMOTE:  ", (accuracy_score(target_test, BernoulliNB)))

lr_model = logistic_regression.fit(data_train, target_train)
pred_lr = lr_model.predict(data_test)
print("linear regression accuracy after SMOTE:  ", (accuracy_score(target_test, pred_lr)))

results['LinearSVC']['balanced'] =accuracy_score(target_test, pred)
results['random forest']['balanced'] =accuracy_score(target_test, rf_pred)
results['knn']['balanced'] =accuracy_score(target_test, predNN)
results['DecisionTree']['balanced'] =accuracy_score(target_test, predDT)
results['BernoulliNB']['balanced'] =accuracy_score(target_test, BernoulliNB)
results['Logistic Regression']['balanced'] =accuracy_score(target_test, pred_lr)

In [ ]:
# Kros-validacija
cv_results = cross_validate(rf_model, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("Random forest cross validation accuracy: ", cv_results['test_score'])
results['random forest']['cross'] =  np.mean(cv_results['test_score'])


cv_results = cross_validate(knn_model, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("KNeighbors cross validation accuracy: ", cv_results['test_score'])
results['knn']['cross'] =  np.mean(cv_results['test_score'])

cv_results = cross_validate(logistic_regression, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("LogisticRegression cross validation accuracy: ", cv_results['test_score'])
results['Logistic Regression']['cross'] = np.mean(cv_results['test_score'])

cv_results = cross_validate(svc_model, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("LinearSVC cross validation accuracy: ", cv_results['test_score'])
results['LinearSVC']['cross'] =  np.mean(cv_results['test_score'])


cv_results = cross_validate(nb_classifer, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("BernoulliNB cross validation accuracy: ", cv_results['test_score'])
results['BernoulliNB']['cross'] = np.mean(cv_results['test_score'])


cv_results = cross_validate(gnb_classifer, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("GaussianNB cross validation accuracy: ", cv_results['test_score'])
results['GaussianNB']['cross'] = np.mean(cv_results['test_score'])

cv_results = cross_validate(decisionTree, data_features, target, cv=kfold, scoring='accuracy')
print("Average Accuracy: ", np.mean(cv_results['test_score']))
print("DecisionTree cross validation accuracy: ", cv_results['test_score'])
results['DecisionTree']['cross'] = np.mean(cv_results['test_score'])

In [344]:
pd.DataFrame.from_dict(results)

,knn,random forest,Logistic Regression,LinearSVC,BernoulliNB,GaussianNB,DecisionTree
basic,0.797872,0.808511,0.808511,0.808511,0.797872,0.202128,0.712766
cross,0.838298,0.846809,0.831915,0.838298,0.834043,0.159574,0.768085
balanced,0.912500,0.912500,0.725000,0.718750,0.675000,0.000000,0.837500
importance,NaN,0.776596,NaN,NaN,NaN,NaN,NaN
